# 機械学習入門_AIプロジェクト演習 (Python)

## 前提（作業環境）

データベースはPostgres10、機械学習環境はAnaconda Python3系にて事前の環境構築をお願いします。DB環境は使い慣れたものがあれば代替していただいても構いませんが、回答コード例は当該環境用のもののみの準備となります。
- Python3系のAnaconda
- Postgres10（pgAdmin4を使用）

In [1]:
import pandas as pd
import numpy as np

## 1. データの取り込みと件数確認

<b>【演習1】</b><br>クライアントからe-commerceの購買取引データを受領し、優良顧客リテンションのための予測モデル構築のフィージビリティ検証を行うことになった。「input」フォルダ内の「ec_trans.csv」データを、作業用に準備したデータベースへテーブル名「raw_ec_trans」で取り込み、データ件数を確認せよ。ただし、クライアントはテーブル定義書を有していないとのことで、データ型は一旦全てvarchar(50)として取り込むものとする。カラム構成と名称はCSVファイルの1行目で確認できる。

In [2]:
ec_trans = pd.read_csv("01_input/ec_trans.csv")
ec_trans.shape

(541909, 8)

In [3]:
ec_trans.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
ec_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## 2. データ確認とクレンジング

<b>【演習2.1】</b><br>データの中身を確認しクライアントに問い合わせたところ、以下の回答を得た。
- StockCodeの小文字と大文字の違いに意味はない
- StockCode5桁目までで1つの商品類型を表し6桁目以降は色やサイズ違いを表している
- InvoiceDateの日付は"MM/DD/YYYY"で並んでいる

上記を踏まえ、以下を満たす「v_ec_trans」という名称のViewを作成せよ。
- 生データのStockCodeをStockCode_orgと名称変更する
- 生データのStockCodeを全て大文字に変換したカラムを作成し、名称をStockCodeとする
- 大文字変換されたStockCodeの左5桁を、StockCode2というカラムとする
- カラムQuantityとUnitPriceのデータ型を、float型に変換する
- Invoiceのデータをタイムスタンプに変換する

In [5]:
v_ec_trans = ec_trans.copy()    ## コピーの作成

In [6]:
v_ec_trans["StockCode_org"] = v_ec_trans["StockCode"]    ## 元のStockCodeをoriginalとしてキープ

In [7]:
v_ec_trans["StockCode"] = v_ec_trans["StockCode_org"].str.upper()    ## 小文字を大文字に変換

In [8]:
v_ec_trans["StockCode2"] = v_ec_trans["StockCode"].str[0:5]    ## 左5桁の切り出し

In [9]:
v_ec_trans["Quantity"] = v_ec_trans["Quantity"].astype("float64")    ## 型変更

In [10]:
v_ec_trans["InvoiceDate"] = pd.to_datetime(v_ec_trans["InvoiceDate"])    ## pandas内での時間型（datetime64）に変換

In [11]:
v_ec_trans.info()    ## 型確認

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 10 columns):
InvoiceNo        541909 non-null object
StockCode        541909 non-null object
Description      540455 non-null object
Quantity         541909 non-null float64
InvoiceDate      541909 non-null datetime64[ns]
UnitPrice        541909 non-null float64
CustomerID       406829 non-null float64
Country          541909 non-null object
StockCode_org    541909 non-null object
StockCode2       541909 non-null object
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 41.3+ MB


In [12]:
v_ec_trans = v_ec_trans[["InvoiceNo","StockCode_org","StockCode","StockCode2",
                         "Description","Quantity","InvoiceDate","UnitPrice",
                         "CustomerID","Country"]]                                    ## カラム順序の並べ替え

In [13]:
v_ec_trans.head(10)

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,85123A,85123,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,71053,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,84406B,84406,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,84029G,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,84029E,84029,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,22752,22752,SET 7 BABUSHKA NESTING BOXES,2.0,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,21730,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,22633,22633,HAND WARMER UNION JACK,6.0,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,22632,22632,HAND WARMER RED POLKA DOT,6.0,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,84879,84879,ASSORTED COLOUR BIRD ORNAMENT,32.0,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


<b>【演習2.2】</b><br>受領したデータの取引期間（InvocieDateの最小値と最大値）を確認せよ。

In [14]:
v_ec_trans["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [15]:
v_ec_trans["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

<b>【演習2.3】</b><br>CustomerIDが非NULLのデータを対象に、データ件数、CustomerIDのユニーク件数、InvoiceNoのユニーク件数を確認せよ。

In [16]:
v_ec_trans["CustomerID"].isnull().sum()     ## nullの数

135080

In [17]:
v_ec_trans["CustomerID"].notnull().sum()     ## 非nullの数

406829

In [18]:
v_ec_trans.loc[v_ec_trans["CustomerID"].notnull(), :].head()        ## CustomerIDが非nullのデータ

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,85123A,85123,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,71053,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,84406B,84406,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,84029G,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,84029E,84029,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [19]:
len(v_ec_trans.loc[v_ec_trans["CustomerID"].notnull(), "CustomerID"].unique())    ## CustomerIDのユニーク数

4372

In [20]:
len(v_ec_trans.loc[v_ec_trans["CustomerID"].notnull(), "InvoiceNo"].unique())    ## InvoiceNoのユニーク数

22190

<b>【演習2.4】</b><br>CustomerIDが非NULLのデータを対象に、購買数量（Quantity）と単価（UnitPrice）の最小値・平均値・最大値を確認せよ。

In [21]:
v_ec_trans.loc[v_ec_trans["CustomerID"].notnull(), ["Quantity","UnitPrice"]].describe()

,Quantity,UnitPrice
count,406829.000000,406829.000000
mean,12.061303,3.460471
std,248.693370,69.315162
min,-80995.000000,0.000000
25%,2.000000,1.250000
50%,5.000000,1.950000
75%,12.000000,3.750000
max,80995.000000,38970.000000


<b>【演習2.5】</b><br>購買数量（Quantity）が負となるケースは注文のキャンセルで生じることがわかった。キャンセルはInvocieNoが"C"で始まるという。CustomerIDが非Nullのデータを対象に、以下2つのケースについて数値確認せよ。
1. InvoiceNoが"C"始まりのときの、Qunatityの最小値・平均値・最大値
1. InvoiceNoが"C"始まりでないときの、Qunatityの最小値・平均値・最大値

In [22]:
## CustomerIDが非Nullのデータを作成しておく

v_ec_trans_custid = v_ec_trans.loc[v_ec_trans["CustomerID"].notnull(), :].copy()
v_ec_trans_custid.shape

(406829, 10)

In [23]:
## InvoiceNoが"C"始まりのデータ
v_ec_trans_custid.loc[v_ec_trans_custid["InvoiceNo"].str.startswith("C"), :].head()

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,D,D,Discount,-1.0,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,35004C,35004,SET OF 3 COLOURED FLYING DUCKS,-1.0,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,22556,22556,PLASTERS IN TIN CIRCUS PARADE,-12.0,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,21984,21984,PACK OF 12 PINK PAISLEY TISSUES,-24.0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,21983,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24.0,2010-12-01 10:24:00,0.29,17548.0,United Kingdom


In [24]:
v_ec_trans_custid.loc[v_ec_trans_custid["InvoiceNo"].str.startswith("C"), :].describe()

,Quantity,UnitPrice,CustomerID
count,8905.000000,8905.000000,8905.000000
mean,-30.859966,18.845519,14991.667266
std,1170.154939,444.366043,1706.772357
min,-80995.000000,0.010000,12346.000000
25%,-6.000000,1.450000,13510.000000
50%,-2.000000,2.950000,14895.000000
75%,-1.000000,4.950000,16393.000000
max,-1.000000,38970.000000,18282.000000


In [25]:
## InvoiceNoが"C"始まりでないデータ
v_ec_trans_custid.loc[~( v_ec_trans_custid["InvoiceNo"].str.startswith("C") ), :].head()

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,85123A,85123,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,71053,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,84406B,84406,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,84029G,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,84029E,84029,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [26]:
v_ec_trans_custid.loc[~( v_ec_trans_custid["InvoiceNo"].str.startswith("C") ), :].describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


<b>【演習2.6】</b><br>CustomerIDが非NULLのデータを対象に、単価（UnitPrice）がゼロのデータ件数を確認せよ。

In [27]:
sum( v_ec_trans_custid["UnitPrice"] == 0 )

40

<b>【演習2.7】</b><br>ここまでの結果を踏まえ、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「1ページ」を完成させよ。

## 3. 予測用データマート設計に関する集計

<b>【演習3.1】</b><br>クライアントによると現在、半年間における購買回数(InvoiceNoのユニーク数)が4回以上か未満かで、顧客ランクを管理しているという。2011年1月から6月末までの半年間のデータを対象に、購買回数4回以上の顧客層と4回未満の顧客層それぞれについて、顧客数、購買金額(Quantity*Unitprice)、購買回数を集計せよ。<br><br>ただし集計はCustomerIDが非NULL、UnitPriceが正のデータを対象に行うこと。また集計結果は、作業用エクセル「優良」シートの黄色セル部分に反映させ後、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「2ページ」を完成せよ。

In [28]:
## UnitPriceが正のデータを作成しておく

v_ec_trans_custid_up = v_ec_trans_custid.loc[v_ec_trans_custid["UnitPrice"]>0, :].copy()
v_ec_trans_custid_up.shape

(406789, 10)

In [29]:
## さらに、購買金額(Quantity*Unitprice)カラムを作成しておく
v_ec_trans_custid_up["PurchaseAmount"] = v_ec_trans_custid_up["Quantity"]*v_ec_trans_custid_up["UnitPrice"]
v_ec_trans_custid_up.shape

(406789, 11)

In [30]:
## InvoiceNoのユニーク数（購買発生回数）を確認
len(v_ec_trans_custid_up["InvoiceNo"].unique())

22186

In [31]:
v_ec_trans_custid_up.head()

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PurchaseAmount
0,536365,85123A,85123A,85123,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,71053,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,84406B,84406,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,84029G,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,84029E,84029,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [32]:
## 2011年1月から6月末までの半年間のデータ
t_start = "2011-01-01 00:00:00"
t_end = "2011-07-01 00:00:00"

tf_date = (v_ec_trans_custid_up["InvoiceDate"] >= t_start) & (v_ec_trans_custid_up["InvoiceDate"] < t_end)
sum(tf_date)   ## 対象データ数

150029

In [33]:
v_ec_trans_custid_up[tf_date].head()

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PurchaseAmount
42481,539993,22386,22386,22386,JUMBO BAG PINK POLKADOT,10.0,2011-01-04 10:00:00,1.95,13313.0,United Kingdom,19.5
42482,539993,21499,21499,21499,BLUE POLKADOT WRAP,25.0,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.5
42483,539993,21498,21498,21498,RED RETROSPOT WRAP,25.0,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,10.5
42484,539993,22379,22379,22379,RECYCLING BAG RETROSPOT,5.0,2011-01-04 10:00:00,2.10,13313.0,United Kingdom,10.5
42485,539993,20718,20718,20718,RED RETROSPOT SHOPPER BAG,10.0,2011-01-04 10:00:00,1.25,13313.0,United Kingdom,12.5


In [34]:
## CustomerID単位のデータ（一行一CustomerID）の作成
cust_dt = v_ec_trans_custid_up[tf_date].groupby("CustomerID").agg({"InvoiceNo":["count","nunique"], "PurchaseAmount":"sum"})
cust_dt = cust_dt.reset_index()
cust_dt.columns = ["CustomerID", "Counts", "InvCounts", "PurchaseAmount"]

#Counts - 履歴データ行数
#InvCounts - 購買回数
#PurchaseAmount - 購買合計金額

cust_dt.head()

,CustomerID,Counts,InvCounts,PurchaseAmount
0,12346.0,2,2,0.00
1,12347.0,71,3,1494.16
2,12348.0,11,2,594.44
3,12350.0,17,1,334.40
4,12352.0,48,8,601.18


In [35]:
cust_dt.shape

(2752, 4)

In [36]:
## 優良顧客
print( cust_dt.loc[cust_dt["InvCounts"]>=4, :].describe() )
print( "Sum of PurchaseAmount" )
print( cust_dt.loc[cust_dt["InvCounts"]>=4, "PurchaseAmount"].sum() )
print( "Sum of InvCounts" )
print( cust_dt.loc[cust_dt["InvCounts"]>=4, "InvCounts"].sum() )

#顧客数  -  717
#購買金額(Quantity*Unitprice)  -  2155242.749988
#購買回数  -  5716

         CustomerID       Counts   InvCounts  PurchaseAmount
count    717.000000   717.000000  717.000000      717.000000
mean   15233.503487   123.539749    7.972106     3005.917364
std     1754.452648   170.516231    7.553667     7691.920729
min    12352.000000     4.000000    4.000000      -71.460000
25%    13725.000000    50.000000    4.000000      864.350000
50%    15140.000000    79.000000    6.000000     1481.540000
75%    16775.000000   141.000000    8.000000     2574.690000
max    18283.000000  2477.000000   78.000000   127365.230000
Sum of PurchaseAmount
2155242.75
Sum of InvCounts
5716


In [37]:
## 優良顧客
print( cust_dt.loc[cust_dt["InvCounts"]<4, :].describe() )
print( "Sum of PurchaseAmount" )
print( cust_dt.loc[cust_dt["InvCounts"]<4, "PurchaseAmount"].sum() )
print( "Sum of InvCounts" )
print( cust_dt.loc[cust_dt["InvCounts"]<4, "InvCounts"].sum() )

#顧客数  -  2035
#購買金額(Quantity*Unitprice)  -  1018654.4809999999
#購買回数  -  3280

         CustomerID       Counts    InvCounts  PurchaseAmount
count   2035.000000  2035.000000  2035.000000     2035.000000
mean   15297.142015    30.197052     1.611794      500.567313
std     1705.953247    32.831263     0.745790      601.286387
min    12346.000000     1.000000     1.000000    -1591.200000
25%    13820.000000    10.000000     1.000000      180.475000
50%    15257.000000    20.000000     1.000000      328.160000
75%    16756.500000    39.000000     2.000000      615.720000
max    18287.000000   593.000000     3.000000     9861.380000
Sum of PurchaseAmount
1018654.4809999999
Sum of InvCounts
3280


<b>【演習3.2】</b><br>2010年12月1日～2011年5月末日の6か月間における優良顧客（購買回数4回以上の顧客）のうち、2011年6月1日～2011年8月末日までの3ヵ月間に購買が1回以上ある人数（リピート人数）を集計せよ。同様に、全顧客（購買回数1回以上）についてもリピート人数を集計せよ。<br><br>ただし集計はCustomerIDが非NULL、UnitPriceが正のデータを対象に行うこと。また集計結果は、作業用エクセルの「トレンド」シートの黄色セル部分に反映させのち、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「3ページ」を完成せよ。

In [38]:
## 2010年12月1日～2011年5月末日の6か月間における優良顧客（購買回数4回以上の顧客）

## 2010年12月から2011年5月末までの半年間のデータ
t_start2 = "2010-12-01 00:00:00"
t_end2 = "2011-06-01 00:00:00"
tf_date2 = (v_ec_trans_custid_up["InvoiceDate"] >= t_start2) & (v_ec_trans_custid_up["InvoiceDate"] < t_end2)
print("対象元データ")
print(v_ec_trans_custid_up[tf_date2].shape)

## 購買回数（ユニークInvoiceNo）
cust_dt2 = v_ec_trans_custid_up[tf_date2][["CustomerID","InvoiceNo"]].groupby("CustomerID").nunique()
cust_dt2 = cust_dt2.drop("CustomerID", axis=1)
cust_dt2 = cust_dt2.reset_index()
cust_dt2 = cust_dt2.rename(columns={"InvoiceNo":"InvCounts"})
print("顧客データ")
print(cust_dt2.shape)
print(cust_dt2.head())

## 優良
print("優良顧客データ")
print(cust_dt2.loc[cust_dt2["InvCounts"]>=4, :].shape)
print(cust_dt2.loc[cust_dt2["InvCounts"]>=4, :].head())


## 分析対象人数  -  714

対象元データ
(149040, 11)
顧客データ
(2767, 2)
   CustomerID  InvCounts
0     12346.0          2
1     12347.0          3
2     12348.0          3
3     12350.0          1
4     12352.0          8
優良顧客データ
(714, 2)
    CustomerID  InvCounts
4      12352.0          8
19     12383.0          5
24     12395.0          8
29     12408.0          4
31     12413.0          4


In [39]:
## 分析対象の優良顧客（2010年12月1日～2011年5月末日の6か月間における購買回数4回以上の顧客）IDを保存
cust_id_analysis = cust_dt2.loc[cust_dt2["InvCounts"]>=4, "CustomerID"]
cust_id_analysis.head()

4     12352.0
19    12383.0
24    12395.0
29    12408.0
31    12413.0
Name: CustomerID, dtype: float64

In [40]:
## 分析対象の優良顧客の内、2011年6月1日～2011年8月末日までの3ヵ月間に購買が1回以上ある顧客

## 2011年6月から2011年8月末までの3か月間のデータ
t_start3 = "2011-06-01 00:00:00"
t_end3 = "2011-09-01 00:00:00"
tf_date3 = (v_ec_trans_custid_up["InvoiceDate"] >= t_start3) & (v_ec_trans_custid_up["InvoiceDate"] < t_end3)
print("対象元データ")
print(v_ec_trans_custid_up[tf_date3].shape)

print("うち、cust_id_analysisにCustIDが入っているデータ")
print(v_ec_trans_custid_up[tf_date3].loc[v_ec_trans_custid_up[tf_date3]["CustomerID"].isin(cust_id_analysis), :].shape)

## その顧客ID
cust_id_analysis_tgt = v_ec_trans_custid_up[tf_date3].loc[v_ec_trans_custid_up[tf_date3]["CustomerID"].
                                                          isin(cust_id_analysis), "CustomerID"].unique()
print("その顧客IDの数")
print(len(cust_id_analysis_tgt))

対象元データ
(82992, 11)
うち、cust_id_analysisにCustIDが入っているデータ
(40011, 11)
その顧客IDの数
580


In [41]:
## 分析対象優良顧客  -  714
# cust_id_analysis

## うち、リピート顧客  -  580
# cust_id_analysis_tgt

<b>【演習3.3】</b><br>StockCode2別の購買回数を集計し降順に表示し、作業用エクセルのシート「カテゴリ別」に結果を貼り付け、上位カテゴリを確認せよ。集計期間は2010年12月1日～2011年11月末日の1年間とする。ただし集計はCustomerIDが非NULL、UnitPriceが正のレコードを対象とすること。

In [42]:
## 2010年12月から2011年11月末までの12か月間のデータ
t_start = "2010-12-01 00:00:00"
t_end = "2011-12-01 00:00:00"
tf_date = (v_ec_trans_custid_up["InvoiceDate"] >= t_start) & (v_ec_trans_custid_up["InvoiceDate"] < t_end)

print("対象元データ")
print(v_ec_trans_custid_up[tf_date].shape)

対象元データ
(389128, 11)


In [43]:
v_ec_trans_custid_up[tf_date]["InvoiceDate"].min()

Timestamp('2010-12-01 08:26:00')

In [44]:
v_ec_trans_custid_up[tf_date]["InvoiceDate"].max()

Timestamp('2011-11-30 17:42:00')

In [45]:
## StockCode2別の購買回数（InvoiceNoのユニーク数）
v_ec_trans_custid_up[tf_date].groupby("StockCode2").agg({"InvoiceNo":"nunique"}).sort_values("InvoiceNo", ascending=False)

,InvoiceNo
StockCode2,
85099,2064
85123,1961
22423,1832
47566,1534
84879,1343
20725,1295
22720,1184
POST,1143
23203,1060


## 4.データマートの作成と予測モデルの構築評価

<b>【演習4.1】</b><br>以下を確認し、モデル用データマート作成用のクエリを完成させ、CSVファイルで出力せよ。
- 作業用エクセルのシート「設計」で、各データマートの特徴量計算期間、正解ラベルの定義期間を確認せよ。
- 作業用エクセルのシート「特徴量」で、特徴量定義を確認せよ。
- CSVの最初の列をCustomerID、最終の列を正解ラベルとして出力せよ。

In [46]:
## "期間"と"購買回数"を指定して、対象顧客IDを返す関数（上で実行した操作の関数化）
def get_custId(start, end, n, df=v_ec_trans_custid_up):

    tf = (df["InvoiceDate"] >= start) & (df["InvoiceDate"] < end)
    print("対象元データ")
    print(df[tf].shape)

    ## 購買回数（ユニークInvoiceNo）
    tf_d = df[tf][["CustomerID","InvoiceNo"]].groupby("CustomerID").nunique()
    tf_d = tf_d.drop("CustomerID", axis=1)
    tf_d = tf_d.reset_index()
    tf_d = tf_d.rename(columns={"InvoiceNo":"InvCounts"})
    print("顧客データ")
    print(tf_d.shape)
    #print(tf_d.head())

    ## 優良
    print("回数フィルタリング顧客データ")
    print(tf_d.loc[tf_d["InvCounts"]>=n, :].shape)
    #print(tf_d.loc[tf_d["InvCounts"]>=n, :].head())
    
    ## 分析対象の顧客IDを返す
    return tf_d.loc[tf_d["InvCounts"]>=n, "CustomerID"]

In [47]:
## モデリング用、特徴量期間CustomerID
id_model = get_custId("2011-01-01 00:00:00", "2011-07-01 00:00:00", 4)
id_model.shape         ## "2011-01-01 00:00:00"<=, <"2011-07-01 00:00:00"で購買回数4回以上

対象元データ
(150029, 11)
顧客データ
(2752, 2)
回数フィルタリング顧客データ
(717, 2)


(717,)

In [48]:
## モデリング用、判定期間CustomerID
id_model_tgt = get_custId("2011-07-01 00:00:00", "2011-10-01 00:00:00", 1)
tf_id = id_model_tgt.isin(id_model)
id_model_tgt = id_model_tgt[tf_id] 
id_model_tgt.shape         ## "2011-01-01 00:00:00"<=, <"2011-07-01 00:00:00"で購買回数4回以上かつ、"2011-07-01 00:00:00"<=, <"2011-10-01 00:00:00"に購買あり

対象元データ
(95976, 11)
顧客データ
(2196, 2)
回数フィルタリング顧客データ
(2196, 2)


(595,)

In [49]:
## モデリング用、対象顧客履歴データ
df_tran_model = v_ec_trans_custid_up.loc[v_ec_trans_custid_up["CustomerID"].isin(id_model),:]

tf = (df_tran_model["InvoiceDate"] >= "2011-01-01 00:00:00") & (df_tran_model["InvoiceDate"] < "2011-07-01 00:00:00")  ## 期間を合わせる
df_tran_model = df_tran_model[tf]
df_tran_model.shape

(88578, 11)

In [50]:
df_tran_model.head()

,InvoiceNo,StockCode_org,StockCode,StockCode2,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,PurchaseAmount
42514,540002,21911,21911,21911,GARDEN METAL SIGN,12.0,2011-01-04 10:23:00,1.65,16656.0,United Kingdom,19.80
42515,540002,22625,22625,22625,RED KITCHEN SCALES,8.0,2011-01-04 10:23:00,8.50,16656.0,United Kingdom,68.00
42516,540002,21259,21259,21259,VICTORIAN SEWING BOX SMALL,24.0,2011-01-04 10:23:00,4.95,16656.0,United Kingdom,118.80
42517,540002,21790,21790,21790,VINTAGE SNAP CARDS,144.0,2011-01-04 10:23:00,0.72,16656.0,United Kingdom,103.68
42518,540003,84970S,84970S,84970,HANGING HEART ZINC T-LIGHT HOLDER,12.0,2011-01-04 10:37:00,0.85,16875.0,United Kingdom,10.20


In [51]:
## モデリング用1行1ユーザーデータの作成

In [52]:
## Groupby操作。標準関数のみ
cust_df = df_tran_model.groupby("CustomerID").agg({"InvoiceNo":"nunique",
                                                   "PurchaseAmount":"sum", 
                                                   "Quantity":"sum",
                                                   "InvoiceDate":["min","max"]})
## インデックス処理とカラム名変更
cust_df = cust_df.reset_index()
cust_df.columns = ["CustomerID","trips","amount_of_yen","quantity","period_begin","period_end"]
cust_df.head()

,CustomerID,trips,amount_of_yen,quantity,period_begin,period_end
0,12352.0,8,601.18,188.0,2011-02-16 12:33:00,2011-03-22 16:08:00
1,12359.0,4,3459.93,961.0,2011-01-12 12:43:00,2011-06-03 12:26:00
2,12383.0,5,1238.59,775.0,2011-01-12 14:10:00,2011-06-08 08:02:00
3,12395.0,7,1035.90,837.0,2011-01-05 11:21:00,2011-06-30 16:52:00
4,12408.0,6,1460.79,725.0,2011-03-24 14:46:00,2011-06-29 15:38:00


In [60]:
cust_df["period"] = (cust_df["period_end"] - cust_df["period_begin"]).dt.days     ## period
cust_df["recency"] = (pd.to_datetime("2011-07-01 00:00:00") - cust_df["period_end"]).dt.days    ## recency
cust_df["amount_of_yen_per_trips"] = cust_df["amount_of_yen"] / cust_df["trips"]    ## amount_of_yen_per_trips
cust_df["regularity"] = cust_df["period"] / cust_df["trips"]    ## regularity

cust_df.head()

,CustomerID,trips,amount_of_yen,quantity,period_begin,period_end,period,recency,amount_of_yen_per_trips,regularity
0,12352.0,8,601.18,188.0,2011-02-16 12:33:00,2011-03-22 16:08:00,34,100,75.147500,4.250000
1,12359.0,4,3459.93,961.0,2011-01-12 12:43:00,2011-06-03 12:26:00,141,27,864.982500,35.250000
2,12383.0,5,1238.59,775.0,2011-01-12 14:10:00,2011-06-08 08:02:00,146,22,247.718000,29.200000
3,12395.0,7,1035.90,837.0,2011-01-05 11:21:00,2011-06-30 16:52:00,176,0,147.985714,25.142857
4,12408.0,6,1460.79,725.0,2011-03-24 14:46:00,2011-06-29 15:38:00,97,1,243.465000,16.166667


In [68]:
## キャンセル回数のカウントデータ

## InvoiceNoがC始まりの履歴データを抜き出し、CustomerIDでGroupby。InvoiceNoのユニーク数
cust_df2 = df_tran_model.loc[df_tran_model["InvoiceNo"].str.startswith("C"),:].groupby("CustomerID").agg({"InvoiceNo":"nunique"})
#df_tran_model.loc[df_tran_model["Quantity"]<0,:].groupby("CustomerID").agg({"InvoiceNo":"nunique"})
cust_df2.head()

## 注：この操作では、SQLでの解答例と数字が合わない

,InvoiceNo
CustomerID,
12352.0,3
12359.0,1
12383.0,1
12395.0,2
12408.0,3


In [71]:
#df_tran_model.loc[df_tran_model["InvoiceNo"].str.startswith("C"),:].describe()

In [72]:
## カテゴリ別購買数データの作成

## InvoiceNoユニークのカウントのため、"CustomerID"と"StockCode2"でGroupBy
df_tran_model_categ = df_tran_model.groupby(["CustomerID","StockCode2"]).agg({"InvoiceNo":"nunique"}).reset_index()
df_tran_model_categ.head()

,CustomerID,StockCode2,InvoiceNo
0,12352.0,21232,1
1,12352.0,21380,1
2,12352.0,21700,1
3,12352.0,21754,1
4,12352.0,21755,1


In [73]:
## StockCode2カテゴリ頻度の作成
categ = ["85099","85123","22423","47566","84879","20725","22720","POST","23203","22383"]
cust_df3 = pd.crosstab(df_tran_model_categ["CustomerID"], df_tran_model_categ["StockCode2"], 
                       values=df_tran_model_categ["InvoiceNo"], aggfunc="sum")[categ].fillna(0)

## カラム名変更
categ_new = ["trips_cat"+str(i) for i in categ]
cust_df3.columns = categ_new

cust_df3 = cust_df3.reset_index()
cust_df3.head()

,CustomerID,trips_cat85099,trips_cat85123,trips_cat22423,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catPOST,trips_cat23203,trips_cat22383
0,12352.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1,12359.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,12383.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3,12395.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
4,12408.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [74]:
## 各CustomerIDにおけるStockCode2の最頻値（InvoiceNoのユニークは取っていない）

df_sc_count = df_tran_model.groupby("CustomerID")["StockCode2"].apply(lambda x: x.mode()).reset_index()  ## 複数の最頻値がある顧客もある
df_sc_count = df_sc_count.sort_values(["CustomerID","level_1"], ascending=[True, False])  ##  答えと揃えるため、最後に出現する最頻値の取得
cust_df4 = df_sc_count.groupby("CustomerID").first()

cust_df4 = cust_df4.drop("level_1", axis=1).reset_index()
cust_df4 = cust_df4.rename(columns={"StockCode2":"mode_category"})
cust_df4.head()

,CustomerID,mode_category
0,12352.0,M
1,12359.0,82613
2,12383.0,84997
3,12395.0,POST
4,12408.0,85099


In [88]:
## Target（離反）の作成

df_tgt = pd.DataFrame(id_model_tgt)
df_tgt["non_tgt"] = 1                        ## リピーターフラグ

cust_df5 = pd.merge(pd.DataFrame(id_model), df_tgt, on="CustomerID", how="left").fillna(0)

cust_df5["tgt"] = 1 - cust_df5["non_tgt"]    ## 離反フラグ
cust_df5.head()

,CustomerID,non_tgt,tgt
0,12352.0,1.0,0.0
1,12359.0,0.0,1.0
2,12383.0,0.0,1.0
3,12395.0,1.0,0.0
4,12408.0,1.0,0.0


In [92]:
## 各データを結合し、モデリングデータの完成

In [93]:
print(cust_df.shape)
print(cust_df2.shape)      ## キャンセル発生顧客のみなので行数は少ない
print(cust_df3.shape)
print(cust_df4.shape)
print(cust_df5.shape)

(717, 10)
(537, 1)
(717, 11)
(717, 2)
(717, 3)


In [97]:
dm_for_model = pd.merge(cust_df, cust_df2, on="CustomerID", how="left").fillna(0)
dm_for_model = pd.merge(dm_for_model, cust_df3, on="CustomerID", how="left")
dm_for_model = pd.merge(dm_for_model, cust_df4, on="CustomerID", how="left")
dm_for_model = pd.merge(dm_for_model, cust_df5, on="CustomerID", how="left")
dm_for_model.head()

,CustomerID,trips,amount_of_yen,quantity,period_begin,period_end,period,recency,amount_of_yen_per_trips,regularity,...,trips_cat47566,trips_cat84879,trips_cat20725,trips_cat22720,trips_catPOST,trips_cat23203,trips_cat22383,mode_category,non_tgt,tgt
0,12352.0,8,601.18,188.0,2011-02-16 12:33:00,2011-03-22 16:08:00,34,100,75.147500,4.250000,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,M,1.0,0.0
1,12359.0,4,3459.93,961.0,2011-01-12 12:43:00,2011-06-03 12:26:00,141,27,864.982500,35.250000,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,82613,0.0,1.0
2,12383.0,5,1238.59,775.0,2011-01-12 14:10:00,2011-06-08 08:02:00,146,22,247.718000,29.200000,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,84997,0.0,1.0
3,12395.0,7,1035.90,837.0,2011-01-05 11:21:00,2011-06-30 16:52:00,176,0,147.985714,25.142857,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,POST,1.0,0.0
4,12408.0,6,1460.79,725.0,2011-03-24 14:46:00,2011-06-29 15:38:00,97,1,243.465000,16.166667,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,85099,1.0,0.0


In [98]:
dm_for_model.shape

(717, 24)

In [99]:
## これをdm_for_model.csvとして保存（ただし、解答例ではカラムはエクセルのシート「特徴量」のみ）

In [ ]:
## フォワードテスト用DMも、期間を変え同様の方法で作成。dm_for_fwd.csvとして保存

<b>【演習4.2】</b><br>フォルダ「\work\python」内にある「classifier.py」を完成させよ（対応箇所は計8箇所あります。"[-------]"の箇所を変更して下さい）。またK-Fold法によるモデル評価結果を、作業用ファイルのシート「モデル管理」に記入し、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「8ページ」を完成せよ。

In [108]:
import os
print(os.getcwd())      ## 現在のディレクトリ
print(os.listdir("./02_work/python/"))       ## 実行するファイル'classifier.py'

C:\Users\yuki.shimada\Documents\37_Lecture\Lecture_ML2\exercise\講師用
['classifier.py', 'data', 'kmeans.py', 'memo.txt', 'model', '_data']


In [109]:
## 'classifier.py'のあるディレクトリ内に"data"フォルダを作成し、dm_for_model.csvとdm_for_fwd.csvを格納

## Anaconda Promptで完成させた'classifier.py'を実行
## >python ディレクトリ/classifier.py

In [110]:
## 実行後"data"フォルダ内に以下の2ファイルが作成される
# ***_with_pred.csv'                        : フォワードテスト用に対する予測結果
# ***_feature_importances.csv               : モデル用で作成したモデル（GBC or RFC）の変数重要度

In [111]:
## 実行後"model"フォルダ内にモデルオブジェクトが保存される
# モデル名.pkl

<b>【演習4.3】</b><br>【演習4.2】で評価したモデルでパフォーマンスの高かったモデルの、変数の重要度が出力されたCSV結果を、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「9ページ」に反映させ完成せよ。

## 5. フォーワード検証

<b>【演習5】</b><br>演習4.2でパフォーマンスの高かったモデルによって出力された予測スコアデータ(「_with_pred」で終わるCSVファイル)をDBに取り込み、スコアを降順に10分割し、当該ランクごとに以下を出力するスクリプトを完成させよ。
- スコアランク
- 予測スコアの最小値
- 予測スコアの平均値
- 予測スコアの最大値
- レコード数[A]
- ターゲット（正例）数[B]
- ターゲット（正例）含有率（[B]/[A])

In [114]:
df_pred = pd.read_csv("./02_work/python/data/GBC_001_dm_for_fwd_with_pred.csv")    ## 予測結果データ
print(df_pred.shape)

df_fwd = pd.read_csv("./02_work/python/data/dm_for_fwd.csv")                       ## ターゲット（tgt）取得のためのフォワードテスト用データ
print(df_fwd.shape)

(807, 2)
(807, 21)


In [131]:
df_res = pd.merge(df_pred, df_fwd[["customerid","tgt"]], on="customerid")    ## 正解ラベル（ターゲット）を取得
df_res.head()

,customerid,pred_score,tgt
0,12362,0.116051,0
1,12395,0.326379,0
2,12408,0.183245,0
3,12409,0.062634,1
4,12415,0.037452,0


In [134]:
df_res["rank"] = df_res["pred_score"].rank(ascending=False, method='min')
df_res = df_res.sort_values("pred_score", ascending=False)
df_res.head()

,customerid,pred_score,tgt,rank
754,17874,0.914307,1,1.0
386,15098,0.858565,1,2.0
804,18260,0.852931,1,3.0
180,13632,0.806854,0,4.0
204,13811,0.785944,1,5.0


In [142]:
df_res["decil_rank"] = np.floor(df_res["rank"] / df_res.shape[0] * 10)    ## 最終行があぶれるがSQL解答例に合わせる
df_res.head()

,customerid,pred_score,tgt,rank,decil_rank
754,17874,0.914307,1,1.0,0.0
386,15098,0.858565,1,2.0,0.0
804,18260,0.852931,1,3.0,0.0
180,13632,0.806854,0,4.0,0.0
204,13811,0.785944,1,5.0,0.0


In [146]:
df_res["decil_rank"].value_counts()

8.0     82
7.0     81
5.0     81
3.0     81
2.0     81
1.0     81
6.0     80
4.0     80
0.0     80
9.0     79
10.0     1
Name: decil_rank, dtype: int64

In [168]:
## 全体での離反率
df_res["tgt"].sum() / df_res.shape[0]

0.16852540272614622

In [169]:
## decil_rank別に集計を行う

df_res_decil = df_res.groupby("decil_rank").agg({"pred_score":["min","mean","max","count"], "tgt":"sum"})
df_res_decil.columns = ["min_score", "avg_score", "max_score", "ids", "tgt"]
df_res_decil["precision"] = df_res_decil["tgt"] / df_res_decil["ids"]
df_res_decil

# tgt: 各decilにおける実際の離反数
# precision: tgt/ids

,min_score,avg_score,max_score,ids,tgt,precision
decil_rank,,,,,,
0.0,0.323169,0.520701,0.914307,80,28,0.350000
1.0,0.193420,0.235951,0.311805,81,21,0.259259
2.0,0.148748,0.170004,0.193255,81,17,0.209877
3.0,0.116666,0.130976,0.148538,81,15,0.185185
4.0,0.094676,0.104917,0.116051,80,13,0.162500
5.0,0.073997,0.083907,0.094466,81,16,0.197531
6.0,0.058623,0.064692,0.073668,80,13,0.162500
7.0,0.045829,0.051335,0.058537,81,8,0.098765
8.0,0.037454,0.041613,0.045463,82,4,0.048780


集計結果は、作業用エクセルのシート「FWD検証」に張り付け、「AIジョブカレ_機械学習入門_初回報告書サンプル_演習用.pptx」の「11ページ」を完成させよ。

## 6. スコア上位者プロファイル（クラスタリング学習後に実施）

<b>【演習6】</b><br>クライアントからスコアランク上位20％相当の顧客の特徴を知りたいと要望があった。上位20%とその他顧客層の違い、及び上位20%内部の購買パターン分析を通し、当該顧客の特徴を示した資料を作成せよ。

In [175]:
## 上のdf_res_decil集計データにおいて、decil_rankが0と1の顧客を分析する

df_res_clst = df_res[df_res["decil_rank"].isin([0,1])]
df_res_clst.head()

,customerid,pred_score,tgt,rank,decil_rank
754,17874,0.914307,1,1.0,0.0
386,15098,0.858565,1,2.0,0.0
804,18260,0.852931,1,3.0,0.0
180,13632,0.806854,0,4.0,0.0
204,13811,0.785944,1,5.0,0.0


In [176]:
df_res_clst.shape

(161, 5)

In [206]:
## decil_rankが0と1の顧客の"trips", "amount_of_yen","regularity"（フォワードテスト用データから取得）

#pd.merge(df_res_clst, df_fwd, on="customerid", how="left")[["customerid","pred_score","tgt_x","rank","trips", "amount_of_yen","regularity"]]
df_res_clst_fnl = pd.merge(df_res_clst, df_fwd, on="customerid", how="left")[["customerid","pred_score","tgt_x","trips", "amount_of_yen","regularity"]]
df_res_clst_fnl = df_res_clst_fnl.rename(columns={"tgt_x":"tgt"})
df_res_clst_fnl.head()

,customerid,pred_score,tgt,trips,amount_of_yen,regularity
0,17874,0.914307,1,6,593.10,6.666667
1,15098,0.858565,1,5,649.50,0.000000
2,18260,0.852931,1,4,1276.73,16.750000
3,13632,0.806854,0,4,810.35,17.250000
4,13811,0.785944,1,4,591.75,13.750000


In [207]:
## このデータをdm_for_cluster.csvとしてモデリング時に作成した"data"フォルダへ保存（"pred_score","tgt"はクラスタリングに必要ない）

#df_res_clst_fnl.to_csv("ディレクトリ/data/dm_for_cluster.csv")

In [208]:
## クラスタリングを実行

## kmeans.pyをAnaconda Promptで実行
## >python ディレクトリ/kmeans.py

## **_with_class.csvが"data"フォルダの中に作成される

In [209]:
df_clst = pd.read_csv("./02_work/python/data/dm_for_cluster_with_class.csv")    ## クラスタリング結果データ
print(df_clst.shape)

(161, 2)


In [210]:
df_clst.head()

,customerid,km_class
0,17874,0
1,15098,0
2,18260,0
3,13632,0
4,13811,0


In [211]:
df_clst = pd.merge(df_clst, df_res_clst_fnl, on="customerid", how="left")
df_clst.head()

,customerid,km_class,pred_score,tgt,trips,amount_of_yen,regularity
0,17874,0,0.914307,1,6,593.10,6.666667
1,15098,0,0.858565,1,5,649.50,0.000000
2,18260,0,0.852931,1,4,1276.73,16.750000
3,13632,0,0.806854,0,4,810.35,17.250000
4,13811,0,0.785944,1,4,591.75,13.750000


In [212]:
df_clst["km_class"].value_counts()

0    81
2    63
1    17
Name: km_class, dtype: int64

In [220]:
## km_classでgroupbyし、顧客理解に役立つと考えられる集計を実施

df_clst.groupby("km_class").agg({"customerid":"count", "tgt":"sum", "trips":"mean", "amount_of_yen":"mean", "regularity":"mean",
                                "pred_score":["min","mean","max"]})

customerid tgt     trips amount_of_yen regularity pred_score  \
              count sum      mean          mean       mean        min   
km_class                                                                
0                81  27  4.419753    723.679519  12.228189   0.199575   
1                17   3  6.764706   1504.741765  14.919024   0.195318   
2                63  19  4.238095    707.065873  27.799206   0.193420   

                              
              mean       max  
km_class                      
0         0.472120  0.914307  
1         0.320442  0.670733  
2         0.271093  0.544125